In [1]:
!pip install --upgrade transformers
!pip install --upgrade nltk
!pip install -q evaluate
!pip install bert-score
!pip install -q rouge_score
!pip install -q git+https://github.com/salaniz/pycocoevalcap.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [2]:
import nltk
nltk.__version__

'3.8.1'

In [3]:
import torch

# device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
torch.cuda.device_count()

2

In [5]:
def count_params(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BeitImageProcessor, AutoModel

# Vision
vision_module = 'microsoft/beit-base-patch16-224-pt22k-ft22k'
feature_extractor = BeitImageProcessor.from_pretrained(vision_module)

# Language
language_module = 'luqh/ClinicalT5-base'
tokenizer = T5Tokenizer.from_pretrained(language_module)

2024-05-07 10:38:26.948360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 10:38:26.948495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 10:38:27.078551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import glob
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import pickle

class ImageCLEF(Dataset):
  def __init__(self, tokenizer = None, feature_extractor = None, image_folder = None, data_csv_path = None, one_hot_path = None, cuis_csv_path = None, data=None):

    self.tokenizer = tokenizer
    self.feature_extractor = feature_extractor
    # if data is None all others argument cant be None
    assert (tokenizer is not None
            and feature_extractor is not None
            and image_folder is not None
            and data_csv_path is not None
            and one_hot_path is not None
            and cuis_csv_path is not None) or data is not None, "All other arguments must be passed if data is None!"

    if data is None:
      self.load_data(image_folder, data_csv_path, one_hot_path, cuis_csv_path)
    else:
      self.data = data

  def load_data(self, image_folder, data_csv_path, one_hot_path, cuis_csv_path):
    self.cuis_mapping = pd.read_csv(cuis_csv_path)

    with open(one_hot_path, 'rb') as file:
      one_hot = pickle.load(file)

    self.one_hot = one_hot
    self.data = []

    data_csv = pd.read_csv(data_csv_path)

    # Setup the total feature file
    image_paths = glob.glob(image_folder + '/*')

    for path in tqdm(image_paths):

    # Obtaining Image Id
      image_id = path.split('/')[-1].split('.')[0]

      # Mapping image id with other variables
      # caption = data_csv[data_csv['image_id'] == image_id]['caption'].item()
      caption = data_csv[data_csv['ID'] == image_id]['Caption'].item()

      sample = {
          'image_id': image_id,
          'path' : path,
          'captions': caption,
      }

      self.data.append(sample)

  def __getitem__(self, idx):
    sample = self.data[idx]

    return {
      'image_id': sample['image_id'],
      'path' : sample['path'],
      'captions': sample['captions'],
    }

  def split_data(self, validation_size, random_state=42):

    # Split train and evaluation set
    train_data, val_data = train_test_split(self.data,
                                                 test_size=validation_size,
                                                 random_state=random_state)

    return (ImageCLEF(tokenizer=self.tokenizer, data=train_data),
            ImageCLEF(tokenizer=self.tokenizer, data=val_data))

  def num_classes(self):
    return len(self.one_hot.classes_)

  def get_classes(self):
    return self.one_hot.classes_

  def get_id_name(self, binary_values):

    mapped_values, mapped_name = [], []

    for i, binary in enumerate(eval(binary_values)):
      if binary == 1:
          res = self.one_hot.classes_[i]
          mapped_values.append(res)

          name = self.cuis_mapping[self.cuis_mapping['CUI'] == res]['Canonical name'].item()
          mapped_name.append(name)

    return mapped_values, mapped_name

  def __len__(self):
    return len(self.data)

  def collate_fn(self, batch):

    images = [Image.open(each['path']).convert('RGB') for each in batch]
    # label_encode = [each['label_encode'] for each in batch]

    raw_captions = [each['captions'] for each in batch]
    image_ids = [each['image_id'] for each in batch]

    extracted_images = self.feature_extractor(images = images, return_tensors = 'pt')
    tokenized_captions = self.tokenizer(raw_captions, padding = True, truncation = True, max_length = 128, return_tensors = 'pt')
    # label_encode = pad_sequence(self.transform_tensor(label_encode), batch_first = True, padding_value = self.num_classes())

    sample = {
      'ids' : image_ids,
      'raw_captions' : raw_captions,
      'pixel_values' : extracted_images.pixel_values, # tensor
      'labels' : tokenized_captions.input_ids, # tensor
      'attention_mask' : tokenized_captions.attention_mask
    }

    return sample

In [8]:
data = ImageCLEF(tokenizer = tokenizer,
                feature_extractor = feature_extractor,
                image_folder ='/kaggle/input/testlehuytu2024/valid_images/valid',
                data_csv_path = '/kaggle/input/testlehuytu2024/valid_captions.csv',
                one_hot_path = '/kaggle/input/testlehuytu2024/one_hot (1).pkl',
                cuis_csv_path = '/kaggle/input/testlehuytu2024/cui_mapping.csv',
                )

  0%|          | 0/9972 [00:00<?, ?it/s]

In [9]:
from torch.utils.data import DataLoader
dataloader = DataLoader(data, batch_size=1, shuffle=False, collate_fn=data.collate_fn)

In [10]:
len(dataloader)

9972

In [ ]:
from Captioning_Module import BeIT_T5_Concepts
from Concepts_Module import ConceptDetectionModel

In [13]:
model_concepts = ConceptDetectionModel(pretrained_image_name = vision_module,
                                       num_labels = data.num_classes()
)

model_caption = nn.DataParallel(BeIT_T5_Concepts(vision_module = vision_module,
                         language_module = language_module,
                         device= device,
))

config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/414M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/892M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_flax_pytorch_utils.py:460: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_numpy.cpp:206.)
  pt_model_dict[flax_key] = torch.from_numpy(flax_tensor)
All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from safetensors.torch import load

file_path = "/kaggle/input/testlehuytu2024/model_concepts_checkpoint.safetensors"
with open(file_path, "rb") as f:
    concepts_weight = f.read()

model_concepts.load_state_dict(load(concepts_weight))

model_caption.module.load_state_dict(torch.load('/kaggle/input/testlehuytu2024/State_dict_BeIT5_Epoch_1_4100.pth', map_location = torch.device(device)))

<All keys matched successfully>

In [15]:
from pycocoevalcap.cider.cider import Cider
cider = Cider()

In [16]:
import evaluate
import numpy as np

meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [38]:
def calculate_mean(numbers):
    total = sum(numbers)
    count = len(numbers)
    mean = total / count
    return mean

def compute_metrics(pred_ans, ground_t):

  print(f'Prediction : {pred_ans}')
  print(f'Ground_truth : {ground_t}')
  print('       ')

  # Compute BLEU, ROUGE, METEOR
  bleu1_score = bleu.compute(predictions=pred_ans, references=ground_t, max_order=1)['bleu']
  bleu2_score = bleu.compute(predictions=pred_ans, references=ground_t, max_order=2)['bleu']
  bleu3_score = bleu.compute(predictions=pred_ans, references=ground_t, max_order=3)['bleu']
  bleu4_score = bleu.compute(predictions=pred_ans, references=ground_t, max_order=4)['bleu']
  rouge_score = rouge.compute(predictions=pred_ans, references=ground_t)['rougeL']
  meteor_score = meteor.compute(predictions=pred_ans, references=ground_t)['meteor']

  # Compute CIDER score
  hypotheses_dict = {i: [h] for i, h in enumerate(pred_ans)}
  references_dict = {i: [r] for i, r in enumerate(ground_t)}

  cider_score, _ = cider.compute_score(references_dict, hypotheses_dict)

  # Compute Bert Score
  bert_score = bertscore.compute(predictions=pred_ans, references=ground_t, model_type = 'microsoft/deberta-xlarge-mnli', device = device)
  bert_score_F1_mean = calculate_mean(bert_score['f1'])

  return np.array([bert_score_F1_mean, bleu1_score, bleu2_score, bleu3_score, bleu4_score, rouge_score, meteor_score, cider_score])

In [39]:
def get_answer(preds):
    p = np.where(np.array(preds)>=0.5,1,0)
    return p

def predict_caption(model_caption, model_concepts, pixel_value, tokenizer, convert_logits_concepts, device, max_length = 50):

  pixel_value = pixel_value.to(device)

  # GENERATE CONCEPTS (Follow Model Concepts Forward Function)
  encoded_image = model_concepts.image_encoder(pixel_values = pixel_value,
                                                 return_dict = True,)

  encoded_image = encoded_image['pooler_output'].squeeze()
  output = model_concepts.linear(encoded_image) #batch_size X dim

  # Concepts Logits
  logits = model_concepts.classifier(output)
  preds = torch.sigmoid(torch.Tensor(logits)).detach().cpu().numpy()
  preds = get_answer(preds)

  # Convert Logits To Concepts
  mapped_names = convert_logits_concepts(str(preds.tolist()))[1]
  mapped_names = ', '.join(mapped_names)

  print(f'Predicted Concepts: {mapped_names}')

  # GENERATE CAPTION (Follow Model Caption Forward Function)

  # Encoding Image
  image_features = model_caption.image_encoder(pixel_values = pixel_value).last_hidden_state
  image_features_ = model_caption.layer_norm(image_features)

  # Encoding Concepts
  tokenized_name = tokenizer(mapped_names, padding = True, truncation = True, max_length = 128, return_tensors = 'pt')

  concept_features = model_caption.concepts_encoder(
      input_ids = tokenized_name.input_ids.to(device),
      attention_mask = tokenized_name.attention_mask.to(device)
  ).last_hidden_state # batch_size, num_concepts, 768

  # Concat Image and Concepts Features
  cross_features = model_caption.layer_norm(torch.cat([image_features_, concept_features], dim = 1)) # batch_size, 193 + num_concepts, 768
  cross_mask = torch.cat([torch.ones(image_features_.size(0), image_features_.size(1)), tokenized_name.attention_mask], dim = 1) # batch_size, 193 + num_concepts

  # Generate Answer
  count = 0

  input_ids = torch.tensor([model_caption.captions_decoder.config.decoder_start_token_id]).to(device) # Decoder Start Token

  while count <= max_length:

    decoded_tokens = model_caption.captions_decoder(input_ids = input_ids.unsqueeze(dim = 0),
                        encoder_hidden_states = cross_features,
                        encoder_attention_mask = cross_mask.to(device)
                        ).last_hidden_state # batch_size, answer_length, 768

    output_tokens = model_caption.lm_head(decoded_tokens) # batch_size, answer_length, vocab_size

    tokens_ids = output_tokens.argmax(dim = -1) # batch_size, answer_length

    # if tokens_ids[:, -1] == tokenizer.eos_token_id :
    #   print(f'Found Eos Token in length : {count}')
    #   break

    input_ids = torch.cat([input_ids, tokens_ids[:, -1]], dim = -1)

    count += 1

  res = tokenizer.decode(input_ids, skip_special_tokens = True)
  return res

In [40]:
def predict(model_caption, model_concepts, dataloader, tokenizer, convert_logits_concepts, device = 'cpu'):

  res_ids, res_predictions, res_captions = [], [], []
  total_scores = 0

  model_caption.to(device), model_concepts.to(device)
  model_caption.eval(), model_concepts.eval()

  with torch.no_grad():

    for i, sample in enumerate(tqdm(dataloader)):

      pixel_value = sample['pixel_values']
      image_id = sample['ids']

      predicted_captions = predict_caption(model_caption = model_caption.module,
                                           model_concepts = model_concepts,
                                           pixel_value = pixel_value,
                                           tokenizer = tokenizer,
                                           convert_logits_concepts = convert_logits_concepts,
                                           device = device)
      total_scores += compute_metrics([predicted_captions], sample['raw_captions'])
        
      res_ids.append(image_id[0])
      res_predictions.append(predicted_captions)
      res_captions.append(sample['raw_captions'][0])

  total_scores = total_scores/len(dataloader)
    
  res_score = {
      'bert_score' : total_scores[0],
      'bleu_score' : {
          'BLEU1' : total_scores[1],
          'BLEU2' : total_scores[2],
          'BLEU3' : total_scores[3],
          'BLEU4' : total_scores[4],
      },
      'rouge_score' : total_scores[5],
      'meteor_score' : total_scores[6],
      'cider_score' : total_scores[7]
  }

  return res_ids, res_predictions

In [41]:
convert_logits_concepts = data.get_id_name

scores, ids, captions, predictions = predict(model_caption = model_caption,
                           model_concepts = model_concepts,
                          dataloader = dataloader,
                          tokenizer = tokenizer,
                          convert_logits_concepts = convert_logits_concepts,
                          device = device)

df = pd.DataFrame({'ID': ids, 'Caption': predictions})
df.to_csv('concept_val_run.csv')
torch.save(scores, 'concept_val_scores.pth')

  0%|          | 0/9972 [00:00<?, ?it/s]

Predicted Concepts: Plain x-ray
Prediction : ['Postoperative radiographs Postoperative radiographs showing total hip replacement Postoperative radiographs Postoperative radiographs Postoperative radiographs Postoperative radiographs Postoperative radiographs Postoperative radiographs']
Ground_truth : ['Anteroposterior radiograph of the pelvis where eccentric placement of the metal head can be seen. Signs of failure of the greater trochanter synthesis were also present.']
       


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

Predicted Concepts: X-Ray Computed Tomography
Prediction : ['CT scan showing right sided obstructive lesion (red arrow) CT scan CT scan CT scan CT scan CT scan CT scan CT scan CT scan CT scan CT scan CT scan']
Ground_truth : ['Axial contrasted CT image of larynx, showing left sided glottic versus supraglottic mass.']
       
Predicted Concepts: Magnetic Resonance Imaging
Prediction : ['MRI brain showing asymmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical symmetrical']
Ground_truth : ['Annotated MRI coronal view of T2 3D-DESS WE at the level of an erupted third molar region, (point B) notice the intra-oral separator in black giving clear demarcation of the tongue and lingual mucosa of the mandible. Structures identified—lingual nerve (yellow arrow), lingual gingiva (red arrow), lateral border of tongue (pink arrow) and mylohyoid muscle (white arrow)']
       
Predicted Co

KeyboardInterrupt: 